# Рост проникновения доставки в Авито

В Авито сейчас есть два способа купить товар: связаться напрямую с продавцом или
воспользоваться доставкой. Одной из важных задач является рост проникновения
доставки. По нашим исследованиям это значительно повышает безопасность сделок и
повышает доверие наших пользователей к Авито.

Одним из способов повышения количества клиентов доставки является
информирование пользователей через разные каналы коммуникации: рассылки,
уведомления

Вам предлагается проанализировать результаты теста с изменением критериев отбора
аудитории пользователей для коммуникации: мы приняли решение о том, чтобы
расширить базу пользователей, которую мы информируем о доставке. Насколько у нас
эффективно это получилось?

In [16]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import norm, binom, sem
from scipy.stats import binom_test, t, ttest_ind
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from tqdm import tqdm
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.power import tt_ind_solve_power

In [2]:
data_dtr = pd.read_csv("delivery_test_results.csv")
data_opm = pd.read_csv("orders_paid_mde.csv")

In [4]:
data_dtr.head(20)

,bucket,split_group,metric,metric_value
0,0,control,contacts,714.0
1,0,control,orders_created,358.0
2,0,control,orders_paid,222.0
3,0,test,contacts,729.0
4,0,test,orders_created,376.0
5,0,test,orders_paid,235.0
6,1,control,contacts,776.0
7,1,control,orders_created,361.0
8,1,control,orders_paid,204.0
9,1,test,contacts,662.0


In [27]:
data_opm = data_opm.sort_values(by='bucket_id').reset_index()[['bucket_id', 'orders_paid']]
data_opm.head()

,bucket_id,orders_paid
0,0,9717
1,1,9655
2,2,9490
3,3,9299
4,4,9564


In [26]:
data_opm.shape, data_dtr.bucket.unique().shape

((250, 2), (256,))

In [21]:
mean = data_opm.orders_paid.mean()
std = data_opm.orders_paid.std()

alpha = 0.01 
power = 0.8
result_effect = 0.01
effect_size = (mean * result_effect) / std

In [22]:
sample_vol = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, nobs1=None, ratio=1, alternative='larger')

In [23]:
sample_vol

81.9286348746947

In [33]:
def choose_data(df, p=0.25):
    df = df.iloc[np.random.randint(0, len(df), int(len(df)*p))]
    return df

def get_info(df, alpha, power = None, result_effect = None):
    mean = df.orders_paid.mean()
    std = df.orders_paid.std()
    if result_effect:
        effect_size = (mean * result_effect) / std
        power = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=None, nobs1=len(df), ratio=1, alternative='larger')
        return power
    if power:
        effect_size = tt_ind_solve_power(effect_size=None, alpha=alpha, power=power, nobs1=len(df), ratio=1, alternative='larger')
        result_effect = (effect_size * std) / mean
        return result_effect

In [34]:
steps = np.arange(0.15, 1.0, 0.05)
for p in steps:
    res_ef = get_info(choose_data(data_opm, p), power=0.8, alpha=0.01)
    print(f"Vol is {int(p*100)}% || result effect: {res_ef}")

Vol is 15% || result effect: 0.014569845715299554
Vol is 20% || result effect: 0.013978294143434102
Vol is 25% || result effect: 0.01123502418748494
Vol is 30% || result effect: 0.011320442879498531
Vol is 35% || result effect: 0.008226314300701663
Vol is 40% || result effect: 0.008853531639790874
Vol is 45% || result effect: 0.008009365008991487
Vol is 50% || result effect: 0.0077471007920912605
Vol is 55% || result effect: 0.008382856893735604
Vol is 60% || result effect: 0.007910728666138185
Vol is 65% || result effect: 0.006710272086714129
Vol is 70% || result effect: 0.00617979185467667
Vol is 75% || result effect: 0.006008796806067992
Vol is 80% || result effect: 0.006170123850627027
Vol is 85% || result effect: 0.005899468507060252
Vol is 90% || result effect: 0.00610938719156535
Vol is 95% || result effect: 0.005997493017701448


In [37]:
result_effect = 0.01
steps = np.arange(0.15, 1.0, 0.05)
for p in steps:
    res_ef = get_info(choose_data(data_opm, p), result_effect=result_effect, alpha=0.01)
    print(f"Vol is {int(p*100)}% || power: {res_ef}")

Vol is 15% || power: 0.409673753600492
Vol is 20% || power: 0.6769090383758377
Vol is 25% || power: 0.6285148656574591
Vol is 30% || power: 0.7303157782265919
Vol is 35% || power: 0.7113420654663527
Vol is 40% || power: 0.7915139184311348
Vol is 45% || power: 0.9281099407058437
Vol is 50% || power: 0.9253652840126936
Vol is 55% || power: 0.9444475971149652
Vol is 60% || power: 0.9967520480070949
Vol is 65% || power: 0.9839328292045418
Vol is 70% || power: 0.9927600448822589
Vol is 75% || power: 0.9945088478674375
Vol is 80% || power: 0.9955318006213139
Vol is 85% || power: 0.9976143928606759
Vol is 90% || power: 0.9989513220550548
Vol is 95% || power: 0.998921922383382


## Формализуем гипотезы

Будем смотреть за уже сформированными метриками, такими как:
- Cуммарное количество запросов телефонов и сообщений в мессенджере (contacts); 
- Количество созданных заказов в "Доставке" (orders_created);
- Количество оплаченных заказов в доставке (orders_paid).

Но также предлагается посмотреть на ratio метрики:
- orders_created к contacts, 
- orders_paid к orders_created,
- orders_paid к contacts

Итак, гипотезы формально выглядят так:

$H_0^{(1)}: \mathbb{E}_{test}[contacts] = \mathbb{E}_{control}[contacts]$

$H_0^{(2)}: \mathbb{E}_{test}[orders\_created] = \mathbb{E}_{control}[orders\_created]$

$H_0^{(3)}: \mathbb{E}_{test}[orders\_paid] = \mathbb{E}_{control}[orders\_paid]$

Их проверяем с помощью ttest

Альернативы двухсторонние

И для ratio метрик гипотезы выглядят следующим образом:

$H_0^{(1)}: \mathbb{E}_{test}[orders\_created/contacts] - \mathbb{E}_{control}[orders\_created/contacts] = 0$

$H_0^{(1)}: \mathbb{E}_{test}[orders\_paid/contacts] - \mathbb{E}_{control}[orders\_paid/contacts] = 0$

$H_0^{(1)}: \mathbb{E}_{test}[orders\_paid/orders\_created] - \mathbb{E}_{control}[orders\_paid/orders\_created] = 0$

Их проверяем с бутсрапом, строя доверительный интервал

In [62]:
df_contacts = data_dtr[data_dtr.metric == 'contacts']
df_orders_created = data_dtr[data_dtr.metric == 'orders_created']
df_orders_paid = data_dtr[data_dtr.metric == 'orders_paid']


In [45]:
data_dtr.pivot_table(
    values=('metric_value'),
    index=['split_group', 'metric']
)

metric_value
split_group metric                      
control     contacts          744.738281
            orders_created    325.101562
            orders_paid       205.566406
test        contacts          726.140625
            orders_created    340.527344
            orders_paid       212.937500

In [49]:
t_contacts = ttest_ind(df_contacts[df_contacts.split_group == 'test'].metric_value, df_contacts[df_contacts.split_group == 'control'].metric_value)
t_orders_created = ttest_ind(df_orders_created[df_orders_created.split_group == 'test'].metric_value, df_orders_created[df_orders_created.split_group == 'control'].metric_value)
t_orders_paid = ttest_ind(df_orders_paid[df_orders_paid.split_group == 'test'].metric_value, df_orders_paid[df_orders_paid.split_group == 'control'].metric_value)


print(f'Contacts: {t_contacts}')
print(f'Orders Created: {t_orders_created}')
print(f'Orders Paid: {t_orders_created}')

Contacts: Ttest_indResult(statistic=-5.4527375304803956, pvalue=7.74120674779068e-08)
Orders Created: Ttest_indResult(statistic=4.554365562292695, pvalue=6.5797588502007575e-06)
Orders Paid: Ttest_indResult(statistic=4.554365562292695, pvalue=6.5797588502007575e-06)


Итак, получили:
- Количество контактов в тесте статзначимо меньше, чем в контроле
- Количество созданных заказов в тесте статзначимо больше, чем в контроле
- Количество оплаченных в тесте статзначимо больше, чем в контроле

In [63]:
df_contacts = df_contacts[['bucket', 'split_group', 'metric_value']]
df_contacts.columns = ['bucket', 'split_group', 'contacts']
df_orders_created = df_orders_created[['bucket', 'split_group', 'metric_value']]
df_orders_created.columns = ['bucket', 'split_group', 'orders_created']
df_orders_paid = df_orders_paid[['bucket', 'split_group', 'metric_value']]
df_orders_paid.columns = ['bucket', 'split_group', 'orders_paid']

In [69]:
data_dtr_control = df_contacts[df_contacts.split_group == 'control'].merge(df_orders_created[df_orders_created.split_group == 'control'], on='bucket').merge(df_orders_paid[df_orders_paid.split_group == 'control'], on='bucket')
data_dtr_test = df_contacts[df_contacts.split_group == 'test'].merge(df_orders_created[df_orders_created.split_group == 'test'], on='bucket').merge(df_orders_paid[df_orders_paid.split_group == 'test'], on='bucket')

Посмотрим на ratio метрики

In [77]:
def ratio_created_to_contacts(data):
    return sum(data.orders_created/data.contacts)

def ratio_paid_to_contacts(data):
    return sum(data.orders_paid/data.contacts)

def ratio_paid_to_created(data):
    return sum(data.orders_paid/data.orders_created)

def stat_intervals(stat, alpha = 0.05):
    return np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])

def get_bootstrap_samples(data, 
                          n_samples = 10000, 
                          custom_func = ratio_created_to_contacts):
    return np.array([custom_func(data.iloc[np.random.randint(0, len(data), len(data))]) for _ in range(n_samples)])

In [78]:
smp_c = get_bootstrap_samples(data_dtr_control)
smp_t = get_bootstrap_samples(data_dtr_test)
print(f'For orders_created to contacts: {stat_intervals(smp_t-smp_c)}')

smp_c = get_bootstrap_samples(data_dtr_control, custom_func=ratio_paid_to_contacts)
smp_t = get_bootstrap_samples(data_dtr_test, custom_func=ratio_paid_to_contacts)
print(f'For orders_paid to contacts: {stat_intervals(smp_t-smp_c)}')

smp_c = get_bootstrap_samples(data_dtr_control, custom_func=ratio_paid_to_created)
smp_t = get_bootstrap_samples(data_dtr_test, custom_func=ratio_paid_to_created)
print(f'For orders_paid to orders_created: {stat_intervals(smp_t-smp_c)}')

For orders_created to contacts: [ 6.31390337 10.35796931]
For orders_paid to contacts: [3.26385289 5.63517497]
For orders_paid to orders_created: [-3.51270657 -0.36535736]


Итак, из анализа ratio метрик получили, что:
- Конверсия из контакта в созданный в доставке заказ увеличилась;
- Конверсия из контака в оплаченный в доставке заказ тоже увеличилась;
- Конверсия из созданного заказа в доставке в оплаченный уменьшилась.

Итого, выводы можно сделать следующие:
- Количество контактов статзначимо упала и увеличилось количество созданных и оплаченных заказов в доставке
- Отношение созданных и оплаченных заказов к количеству контактов выросло
- Отношение числа оплаченных заказов в доставке к числу созданных заказов в доставке упало, что говорит о существовании какой-то дополнительной причины, почему пользователи добавили заказ в доставке, но не оплатили его

В целом, то что мы хотели достичь - выполнено